In [2]:
!pip install numpy matplotlib tensorflow

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
pip install kaggle


  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
Using cached packaging-25.0-py3-none-any.whl (66 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflowjs 3.18.0 requires packaging~=20.9, but you have packaging 25.0 which is incompatible.


In [5]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

Dataset URL: https://www.kaggle.com/datasets/anandkumarsahu09/cattle-breeds-dataset
License(s): CC0-1.0
cattle-breeds-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!tar -xf cattle-breeds-dataset.zip


In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [8]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [9]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False


In [10]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 843ms/step - accuracy: 0.2817 - loss: 1.7656 - val_accuracy: 0.4728 - val_loss: 1.3046 - learning_rate: 1.0000e-04
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 432ms/step - accuracy: 0.4830 - loss: 1.2920 - val_accuracy: 0.5983 - val_loss: 1.1503 - learning_rate: 1.0000e-04
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 432ms/step - accuracy: 0.5676 - loss: 1.1173 - val_accuracy: 0.6360 - val_loss: 1.0627 - learning_rate: 1.0000e-04
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 421ms/step - accuracy: 0.6202 - loss: 0.9768 - val_accuracy: 0.6569 - val_loss: 1.0200 - learning_rate: 1.0000e-04
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 418ms/step - accuracy: 0.6656 - loss: 0.8873 - val_accuracy: 0.6820 - val_loss: 0.9528 - learning_rate: 1.0000e-04
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 425ms/step - accuracy: 0.6914 - loss: 0.8300 - val_accuracy: 0.6862 - val_loss: 0.9676 - learning_rate: 1.0000e-04
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 422ms/step - acc

In [20]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [21]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 287ms/step


array([[0.03368833, 0.00397387, 0.0016516 , 0.06381506, 0.89687115],
       [0.24621655, 0.08675801, 0.5691443 , 0.0439771 , 0.05390403],
       [0.06006036, 0.2767382 , 0.46777096, 0.10937123, 0.08605932],
       ...,
       [0.05688354, 0.00787234, 0.906216  , 0.01488502, 0.01414305],
       [0.10192009, 0.03612401, 0.8451707 , 0.0048588 , 0.01192641],
       [0.02481569, 0.14461152, 0.00612904, 0.44014084, 0.3843029 ]],
      shape=(239, 5), dtype=float32)

In [22]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([4, 2, 2, 1, 1, 2, 0, 2, 4, 2, 0, 3, 0, 1, 1, 2, 1, 4, 1, 3, 3, 3,
       4, 0, 4, 1, 3, 2, 0, 0, 0, 1, 3, 1, 2, 2, 3, 3, 1, 0, 1, 4, 1, 1,
       2, 0, 0, 3, 3, 0, 3, 2, 3, 3, 1, 2, 0, 3, 2, 1, 1, 2, 0, 0, 2, 2,
       2, 4, 2, 1, 0, 1, 2, 0, 2, 4, 4, 4, 1, 2, 2, 4, 4, 1, 1, 2, 4, 4,
       3, 4, 0, 4, 0, 0, 0, 0, 1, 3, 2, 0, 3, 4, 2, 2, 4, 4, 4, 1, 1, 2,
       3, 1, 0, 0, 0, 3, 4, 0, 3, 4, 2, 0, 1, 2, 4, 1, 4, 2, 2, 2, 4, 4,
       3, 2, 4, 2, 1, 3, 2, 0, 2, 3, 2, 2, 0, 1, 3, 0, 3, 3, 1, 3, 3, 4,
       1, 0, 2, 3, 4, 1, 1, 3, 0, 1, 3, 4, 2, 0, 4, 1, 4, 0, 0, 1, 1, 1,
       0, 3, 1, 4, 2, 4, 2, 0, 3, 2, 4, 0, 1, 1, 0, 0, 4, 1, 0, 4, 2, 0,
       0, 4, 0, 2, 1, 4, 0, 3, 2, 1, 3, 1, 2, 4, 4, 4, 0, 3, 0, 4, 4, 2,
       2, 0, 4, 2, 3, 3, 1, 4, 0, 0, 1, 1, 0, 4, 1, 4, 2, 2, 3])

In [23]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[11 13 11 11  6]
 [10  9 13  5 10]
 [ 9  7 14  9 11]
 [13 13  6  8 10]
 [ 8  7  8  6 11]]


In [24]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [25]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.22      0.21      0.21        52
      Brown Swiss cattle       0.18      0.19      0.19        47
Holstein Friesian cattle       0.27      0.28      0.27        50
           Jersey cattle       0.21      0.16      0.18        50
         Red Dane cattle       0.23      0.28      0.25        40

                accuracy                           0.22       239
               macro avg       0.22      0.22      0.22       239
            weighted avg       0.22      0.22      0.22       239



In [26]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 296ms/step - accuracy: 0.6862 - loss: 0.9111
Validation Accuracy: 68.62%
